In [2]:
#https://www.nature.com/articles/s41467-018-08090-0  Mode Frequencies/LD params

import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special

def eval_rabi_freq(nStart, nDelta, eta):
    """
    Calculates Rabi Frequency for nStart -> nStart + nDelta

    Args:
        nStart (int): Initial phonon state
        nDelta (int): Change in phonon number
        eta (float): Lamb-Dicke Parameter

    Returns:
        float: Rabi frequency
    """
    nEnd = nStart + nDelta

    nSmall = min([nStart, nEnd])
    nBig = max([nStart, nEnd])
    factor2 = np.exp(-0.5 * eta**2) * eta**(np.absolute(nDelta))
    factor3 = np.sqrt(np.math.factorial(nSmall)/np.math.factorial(nBig))
    factor4 = special.assoc_laguerre(eta**2, nSmall, np.absolute(nDelta))

    return factor2 * factor3 * factor4

def generate_ld_matrix(ld_param):
    """
    Generate 3x3 matrix containing the Lamb-Dicke parameters for 3 ions for the 3 modes of motion along each axis.
    
    Rows label modes
    Columns label the ions
    
    E.g) matrix[1][2]
        Row = 1 -> OP1 Mode
        Column = 2 -> 3rd Ion
    """
    matrix = np.empty([3, 3])
    
    for i in range(3):
        if i == 0: # Center-of-Mass Motion
            for j in range(3):
                matrix[i][j] = ld_param
        elif i == 1: # OP1
            matrix[i][0] = (ld_param * np.sqrt(3))/np.sqrt(2)
            matrix[i][1] = 0
            matrix[i][2] = (ld_param * np.sqrt(3))/np.sqrt(2)
        else:   # OP2
            matrix[i][0] = (ld_param * np.sqrt(3))/np.sqrt(6)
            matrix[i][1] = -2 * (ld_param * np.sqrt(3))/np.sqrt(6)
            matrix[i][2] = (ld_param * np.sqrt(3))/np.sqrt(6)
    
    return matrix

# Max Fock State
N = 10

#### Infidelity Calculations

For a MS interaction involving ground-state cooled motional modes, the infidelity is approximately given by:

$ F = \frac{4}{5} \sum_p (|\alpha_{i,p}|^2 + |\alpha_{j,p}|^2), \quad \alpha_{i, p} = \int^\tau_0 \Omega(t) \sin[\psi(t)] e^{i\omega_p t}, \quad \psi(t) = \int^t_0 \mu(t')dt' $

The index $i$ labels the ions, and $p$ the motional modes, with an associated mode frequency $\omega_p$. Here, $\tau$ is the gate time, and $\Omega(t)$ and $\mu(t')$ are the time-dependent Rabi frequency and detuning respectively.

### Three-ion System Set-up

In [3]:
# --------------------- Rabi Frequenies --------------------
ax_freq = 0.404
r1_freq = 1.093
r2_freq = 1.375

ax_modes = [ax_freq, np.sqrt(3) * ax_freq, np.sqrt(29/5) * ax_freq]
r1_modes = [r1_freq, np.sqrt(r1_freq**2 - ax_freq**2), np.sqrt(r1_freq**2 - (12/5) * ax_freq**2)]
r2_modes = [r2_freq, np.sqrt(r2_freq**2 - ax_freq**2), np.sqrt(r2_freq**2 - (12/5) * ax_freq**2)]

# --------------------- Lamb-Dicke Parameteres --------------------
ax_ld_param = 0.26
r1_ld_param = 0.1
r2_ld_param = 0.1

     

ax_ld = generate_ld_matrix(ax_ld_param)
r1_ld = generate_ld_matrix(r1_ld_param)
r2_ld = generate_ld_matrix(r2_ld_param)

# --------------------- Experimental Parameters --------------------
carr_rabi = 0.3 # Approximate value for 10us pi-time